## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [10]:
import sys
sys.path.append('waveglow')

import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

# print the updated sys.path
print('Updated sys.path:', sys.path)

import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from waveglow.denoiser import Denoiser

Updated sys.path: ['/home/koushik/miniconda3/envs/torch/lib/python311.zip', '/home/koushik/miniconda3/envs/torch/lib/python3.11', '/home/koushik/miniconda3/envs/torch/lib/python3.11/lib-dynload', '', '/home/koushik/miniconda3/envs/torch/lib/python3.11/site-packages', 'waveglow', 'tacotron2', 'waveglow']


In [11]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

#### Setup hparams

In [12]:
hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load model from checkpoint

In [13]:
checkpoint_path = "/mnt/d/Data/tacotron2/checkpoint/checkpoint_6000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [14]:
waveglow_path = '/mnt/d/Data/tacotron2/waveglow/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

#### Prepare text input

In [15]:
text = "Hail, noble cousin. Now, let's get you from that miserable place. I'll see that all of us upon Olympus do our part, beginning here with me."
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

#### Decode text input and plot results

In [16]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
# plot_data((mel_outputs.float().data.cpu().numpy()[0],
#            mel_outputs_postnet.float().data.cpu().numpy()[0],
#            alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [17]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [18]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 